In [109]:
import re
import pandas as pd
import numpy as np
import src.functions as f
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [110]:
db = client.get_database("ironhack")

In [111]:
companies = db.get_collection("companies")

All the companies dedicated to design

All the tech companies that have raised at least 1 Million dollars

In [112]:
query_1 = {'$and' : [{"tag_list" : {'$regex':'design'}}, {"tag_list" : {'$regex':'(?s)^((?!fashion).)*'}}, {"tag_list" : {'$regex':'(?s)^((?!interior).)*'}}]}
query_2 = {'$and' : [{"total_money_raised" : {'$regex':'M'}},{"tag_list" : {'$regex':'tech'}}]}
proj = {'name': 1, '_id': 0, 'total_money_raised' : 1, 'offices.city' : 1, 'offices.latitude': 1, 'offices.longitude': 1, 'offices.zip_code' : 1}
companies_filtered = list(companies.find({'$or' : [query_2, query_1]}, proj))

In [113]:
companies_filtered[0]

{'name': 'Digg',
 'total_money_raised': '$45M',
 'offices': [{'zip_code': '94107',
   'city': 'San Francisco',
   'latitude': 37.764726,
   'longitude': -122.394523}]}

In [115]:
name = []
city = []
latitude = []
longitude = []
zip_code = []
for i in companies_filtered:
    name.append(i['name'])
    try: 
        if i['offices'][0]['city'] == '':
            city.append(np.nan)
        else:
            city.append(i['offices'][0]['city'])
        if i['offices'][0]['zip_code'] == '':
            zip_code.append(np.nan)
        else:
            zip_code.append(i['offices'][0]['zip_code'])
        latitude.append(i['offices'][0]['latitude'])
        longitude.append(i['offices'][0]['longitude'])
    except:
        city.append(np.nan)
        latitude.append(np.nan)
        longitude.append(np.nan)
        zip_code.append(np.nan)

In [116]:
dict_ = {'company' : name, 'city' : city, 'latitude' : latitude, 'longitude': longitude, 'zip_code' : zip_code}

In [117]:
companies_df = pd.DataFrame.from_dict(dict_, orient='columns')

In [118]:
dict_of_cities = f.vc_to_dict(companies_df.city)

In [119]:
cities = f.subdata(11,dict_of_cities)

In [120]:
cities

['San Francisco', 'New York', 'London']

In [121]:
df = f.create(companies_df,'city',cities)

In [122]:
df.sample(4)

,company,city,latitude,longitude,zip_code
740,Ramp Up Tech,New York,40.746514,-73.993035,10001
737,Synesis IT,New York,40.748873,-73.987106,10001
482,SoftFacade,New York,NaN,NaN,10010
322,Box UK,London,51.510774,-0.139245,W1B 4BD


In [125]:
df.to_csv("data/df.csv", index=False)

## Mapitas

In [132]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import src.maps as m

In [133]:
san_francisco = {'type': 'Point', 'coordinates': [37.773972, -122.431297]}
london = {'type': 'Point', 'coordinates': [51.509865, -0.118092]}
new_york = {'type': 'Point', 'coordinates': [40.730610, -73.935242]}

In [134]:
map_sf = Map(location=san_francisco['coordinates'],zoom_start=15)
map_lnd = Map(location=london['coordinates'],zoom_start=15)
map_ny = Map(location=new_york['coordinates'],zoom_start=15)

In [131]:
m.institution(df, 'London', map_lnd)